In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('Dog-Cat-QA.csv')
df.head()

,Unnamed: 0,Question,Answer
0,0,What are some examples of breed-typical person...,Breed-typical personalities in dogs are develo...
1,1,How do working and herding dogs typically behave?,Working and herding dogs have business-like di...
2,2,Which breed of dogs is known for strong loyalty?,Collies and Akitas are known for their strong ...
3,3,What instincts do guarding dogs typically disp...,Guarding dogs tend to be protective of their t...
4,4,How does breed specificity impact a dog's abil...,Breed specificity affects how well dogs adapt ...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  583 non-null    int64 
 1   Question    583 non-null    object
 2   Answer      583 non-null    object
dtypes: int64(1), object(2)
memory usage: 13.8+ KB


In [8]:
df.columns

Index(['Unnamed: 0', 'Question', 'Answer'], dtype='object')

In [9]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


--2024-09-09 12:21:52--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: 'minsearch.py'

     0K ...                                                   100% 12.1M=0s

2024-09-09 12:21:52 (12.1 MB/s) - 'minsearch.py' saved [3832/3832]



In [19]:
import minsearch


In [16]:
documents = df[['Question', 'Answer']].to_dict(orient='records')

# data = df.to_dict(orient='records')


In [17]:
documents[0]

{'Question': 'What are some examples of breed-typical personalities in dogs?',
 'Answer': 'Breed-typical personalities in dogs are developed through generations of selection for certain traits. For example, sporting dogs are adventurous and responsive to humans, while hounds tend to be more independent and aloof.'}

In [21]:
index = minsearch.Index(
     text_fields=["Question", "Answer"],
     keyword_fields=[]
)

In [22]:
q='which cat breed is the most friendly?'

In [23]:
index.fit(documents)


In [26]:
from openai import OpenAI
client = OpenAI()


In [27]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"While individual personalities can vary widely, certain cat breeds are generally known for their friendly and sociable nature. Some of the most commonly cited friendly cat breeds include:\n\n1. **Ragdoll**: Known for their calm and gentle demeanor, Ragdolls often enjoy being held and tend to follow their owners around.\n2. **Maine Coon**: These large cats are known for their friendly disposition and often get along well with both people and other pets.\n3. **Siamese**: Siamese cats are very social and tend to form strong bonds with their owners, often seeking out companionship and interaction.\n4. **British Shorthair**: Known for their easygoing temperament, British Shorthairs are typically affectionate and enjoy spending time with their human families.\n5. **Burmese**: These cats are playful and affectionate, often seeking out human attention and interaction.\n6. **Scottish Fold**: With their distinct folded ears, Scottish Folds are usually friendly and enjoy being around people.\n\n

In [30]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={ },
        boost_dict=boost,
        num_results=5
    )

    return results

In [36]:
query = 'which dog breed is the most friendly?'


In [37]:
search_results=search(query)
print(search_results)

[{'Question': 'Which breed of dogs is known for strong loyalty?', 'Answer': 'Collies and Akitas are known for their strong sense of loyalty.'}, {'Question': 'How can I choose the right dog or breed for my lifestyle?', 'Answer': "Consider factors such as the dog's size, energy level, temperament, and grooming needs. Visiting potential dog candidates in person is also recommended."}, {'Question': ' What is the most important thing to do when your dog is scared of fireworks?', 'Answer': " Staying calm and relaxed is crucial because dogs look to their owners for reassurance. Making a big fuss can reinforce the dog's panic."}, {'Question': 'What is the most advanced obstacle in feline agility training?', 'Answer': 'Weaving around poles is one of the most advanced obstacles in feline agility training.'}, {'Question': " Have you thought about the cat's breed and how it aligns with your lifestyle and preferences?", 'Answer': ' Consider which cat breed best suits your lifestyle and preferences 

In [29]:
from openai import OpenAI
client = OpenAI()


In [47]:
def build_pet_care_prompt(query, search_results):
    prompt_template = """
You are a knowledgeable pet care expert specializing in dogs and cats. 
Answer the USER QUESTION based on the INFORMATION from our pet care database. 
Use only the facts from the INFORMATION when answering the USER QUESTION. 
If the information doesn't provide a complete answer, 
say so and suggest seeking professional veterinary advice.

USER QUESTION: {question}

INFORMATION:
{context}

Please provide a clear and concise answer. If the question is about a specific breed or species, make sure to highlight that in your response. If there are any safety concerns or potential health risks mentioned in the context, emphasize those in your answer.
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Question: {doc['Question']}\nAnswer: {doc['Answer']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [48]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [51]:
len(documents)

583

In [49]:

def rag(query):
    search_results = search(query)
    prompt = build_pet_care_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [50]:
query = 'What traits define the personality of sporting dog breeds'
answer=rag(query)
print(answer)


The information provided does not specify which dog breed is the most friendly. For a definitive answer, it is best to seek professional veterinary advice or consult with a canine behavior specialist.


In [56]:
#rename first column to 'id'


In [57]:
df.head()

,id,Question,Answer
0,0,What are some examples of breed-typical person...,Breed-typical personalities in dogs are develo...
1,1,How do working and herding dogs typically behave?,Working and herding dogs have business-like di...
2,2,Which breed of dogs is known for strong loyalty?,Collies and Akitas are known for their strong ...
3,3,What instincts do guarding dogs typically disp...,Guarding dogs tend to be protective of their t...
4,4,How does breed specificity impact a dog's abil...,Breed specificity affects how well dogs adapt ...


In [58]:
# update df back to file


In [59]:
df=pd.read_csv('Dog-Cat-QA.csv')

In [61]:
df.head(),len(df)

(   id                                           Question  \
 0   0  What are some examples of breed-typical person...   
 1   1  How do working and herding dogs typically behave?   
 2   2   Which breed of dogs is known for strong loyalty?   
 3   3  What instincts do guarding dogs typically disp...   
 4   4  How does breed specificity impact a dog's abil...   
 
                                               Answer  
 0  Breed-typical personalities in dogs are develo...  
 1  Working and herding dogs have business-like di...  
 2  Collies and Akitas are known for their strong ...  
 3  Guarding dogs tend to be protective of their t...  
 4  Breed specificity affects how well dogs adapt ...  ,
 583)

In [62]:
query="What are the signs of stress in cats and dogs?"
answer=rag(query)
print(answer)


Signs of stress in dogs may include excessive yawning, lip-licking, trembling, avoiding or hiding, and a hardening of the eyes. 

For cats, while specific signs of stress are not detailed in the provided information, it is generally known that changes in behavior, loss of appetite, hiding, and changes in grooming habits can also indicate stress in cats.

If you notice signs of stress in your pet, it's important to consult with a veterinarian to rule out any medical issues and address their well-being.


Retrival evaluation